# CIFAR Performance
> Performance data for the CIFAR benchmark

- toc: true 
- badges: true
- comments: true
- categories: [l3, performance, CIFAR]
- image: images/cifar_logo.png

In [1]:
csv_path = "./data/cleaned_csv/backup.csv"

# Theretical Analysis of CIFAR

### Rooflines

The plot below shows.. Lorem ipsum ...

In [6]:
#hide_input
%run scripts/altair_plots.py  #run the heatmaps script
#load dataset and plot it
rooflines(pd.read_csv("data/processed_csv/rooflines_hardware_neural_networks.csv"), 'cifar')

alt.LayerChart(...)

### Performance Prediction

The plot below shows... Lorem ipsum ...

In [7]:
#hide_input
%run scripts/altair_plots.py  #run the heatmaps script
#load cifar10 dataset and plot it
heatmap(pd.read_csv("data/processed_csv/performance_prediction_cifar10.csv"), 'pink', 'Performance prediction for CIFAR 10')

alt.LayerChart(...)

# Experimental Data Analysis

### Overview of All Measurements for CIFAR

In [10]:
#hide_input
%run scripts/altair_plots.py   #get table with the experiments overview
tableOverviewExperiments(['data/overview_experiments_cifar10.csv'])

In [ ]:
#hide
import pandas as pd
import numpy as np
import altair as alt

W = 600
H = 480

In [ ]:
#hide
#%writefile scripts/utils.py
def norm_by_group(df, column, group_col):
    """ Normalizes pandas series by group """
    df["norm-"+column] = df.groupby(group_col)[column].apply(lambda x: (x / x.max()))
    return df

def select_color(sel, column):
    """ Easy way to set colors based on selection for altair plots
    """
    return alt.condition(sel, 
                      alt.Color(column),
                      alt.value('lightgray'))

def get_pareto_df(df, groupcol, xcol, ycol):
    pareto_line_df = df.groupby(groupcol)[xcol].max().to_frame("x")
    pareto_line_df['y'] = df.groupby(groupcol)[ycol].agg(lambda x: x.value_counts().index[0])
    pareto_line_df.sort_values('y', ascending=False, inplace=True)
    pareto_line_df['x'] = pareto_line_df.x.cummax()
    pareto_line_df.drop_duplicates('x', keep='first', inplace=True)
    pareto_line_df['group'] = pareto_line_df.index
    return pareto_line_df

def label_point(x, y, val, ax, rot=0):
    """ from https://stackoverflow.com/questions/46027653/adding-labels-in-x-y-scatter-plot-with-seaborn"""
    a = pd.concat({'x': x, 'y': y, 'val': val}, axis=1)
    for i, point in a.iterrows():
        ax.text(point['x']+.02, point['y'], str(point['val']), rotation=rot)

In [ ]:
#hide
master_df = pd.read_csv(csv_path)
is_maxp = lambda row: row.HWType != "GPU" or row["Op mode"].split(",")[0] == "maxp"
maxp_df = master_df[master_df.apply(is_maxp, axis=1)]
maxp_df["hw_quant_prun"] = maxp_df.apply(lambda r: "_".join([r.HWType, r.Precision, r.PruningFactor]), axis=1)
cnv_df = maxp_df[(maxp_df["NN_Topology"] == "CNV") & maxp_df['lat-comp'].notna() & maxp_df["top1 [%]"].notna()]
cnv_df["hw_quant_prun"] = cnv_df.apply(lambda r: "_".join([r.HWType, r.Precision, r.PruningFactor]), axis=1)
cnv_df["PruningFactor"] = cnv_df["PruningFactor"].str.strip("%").astype(float)
norm_by_group(cnv_df, "lat-comp", "NN_Topology");
cnv_df["quant_model"] = cnv_df.Precision + '_' + cnv_df.HWType
cnv_df.rename(columns={"top1 [%]": "top1"}, inplace=True)

In [ ]:
#hide
figa_df = cnv_df[(cnv_df["HWType"].isin(["NCS", "ZCU104-Bismo", "A53-gemmlowp"]))]
figb_df = cnv_df[(cnv_df["HWType"].isin(["GPU", "ZCU104-FINN", "A53-gemmlowp"]))]

### Line Plot

In [ ]:
#hide_input
fig25s = []
fig25_dfs = [figa_df, figb_df]
for df in fig25_dfs:
    sel = alt.selection_multi(fields=["hw_quant_prun"], bind="legend")
    fig25_dot = alt.Chart(df).mark_point().encode(
        x='lat-comp',
        y=alt.Y('fps-comp', scale=alt.Scale(type="log")),
        color=select_color(sel, 'hw_quant_prun:N'),
        tooltip=['fps-comp', 'lat-comp', 'HWType', 'batch/thread/stream'],
    )
    fig25_line = alt.Chart(df).mark_line().encode(
        x='lat-comp',
        y='fps-comp',
        color=select_color(sel, 'hw_quant_prun:N'),
        tooltip=['fps-comp', 'lat-comp', 'HWType', 'batch/thread/stream'],
    )

    fig = (fig25_dot+fig25_line).properties(
        title="Latency versus Performance for Pruned and Quantized CNV Variants",
        width=W/len(fig25_dfs),
        height=H,
    ).add_selection(sel).interactive()
    
    fig25s.append(fig)
    
alt.hconcat(*fig25s)

### Boxplots

In [ ]:
#hide_input
box1 = alt.Chart(cnv_df).mark_boxplot().encode(
    x='PruningFactor:O',
    y=alt.Y("lat-comp", scale=alt.Scale(type="log")),
    color='PruningFactor:O',
).facet(column="quant_model").properties(
    title="Latency by Hardware/Framework and Pruning for CNV"
).interactive()
box1

In [ ]:
#hide_input
box1 = alt.Chart(cnv_df).mark_boxplot().encode(
    x='PruningFactor:O',
    y=alt.Y("fps-comp", scale=alt.Scale(type="log")),
    color='PruningFactor:O',
).facet(column="quant_model").properties(
    title="Throughput by Hardware/Framework and Pruning for CNV",
).interactive()
box1

### Pareto Graphs

In [ ]:
#hide_input
cnv_pareto = get_pareto_df(cnv_df, 'hw_quant_prun', 'fps-comp', 'top1')

cnv_lines = alt.Chart(cnv_df).mark_line(point=True).encode(
    x="fps-comp",
    y=alt.Y("top1:Q", scale=alt.Scale(zero=False)),
    color=alt.Color("hw_quant_prun", legend=alt.Legend(columns=1)),
    tooltip=["HWType", "Precision", "PruningFactor", "batch/thread/stream", "top1", "fps-comp"],
)
cnv_pareto_plot = alt.Chart(cnv_pareto).mark_line().encode(
    x="x",
    y=alt.Y("y", scale=alt.Scale(zero=False)),
)
(cnv_lines+cnv_pareto_plot).interactive().properties(
    width=W,
    height=H,
    title="cnv Cassification Design Space: Accuracy versus Performance"
)

# Measurements

In [ ]:
#hide_input
cnv_df.describe()